Notebook to try to predict if the exchange rate will increase or decrease given the previous rates

In [ ]:
# imports
import json
import numpy as np
import pandas as pd
import re
import requests

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
# Hide warnings due to new columns for rates
pd.options.mode.chained_assignment = None

In [ ]:
# Get the rates from the API
url = 'https://api.apilayer.com/exchangerates_data/timeseries?start_date=2022-01-09&end_date=2022-11-11&base=GBP&symbols=USD'

payload = {}
headers = {'apikey': 'iVhclvMkTD98sD5kpwCGrD75B5sxuAjK'}

response = requests.request('GET', url, headers=headers, data = payload)
status_code = response.status_code
result = json.loads(response.text)
exch_rates = result['rates']

In [ ]:
# Take the rates from the json
rate_list = []
for key, value  in exch_rates.items():
    text_val = str(value)
    rate_val = re.findall("\d+\.\d+", text_val)
    rate = (key,float(rate_val[0]))
    rate_list.append(rate)

In [ ]:
# Create dataframe and replace change with -1 or 1 to show increase or decrease
full_df = pd.DataFrame (rate_list, columns = ['date', 'rate'])
full_df['change'] = full_df.rate.diff()
full_df['change'] = [-1 if change < 0 else 1 for change in full_df['change']]

In [ ]:
# Create columns to take rates from previous days
full_df['day_1'] = full_df.change.shift(1)
full_df['day_2'] = full_df.day_1.shift(1)
full_df['day_3'] = full_df.day_2.shift(1)
full_df['day_4'] = full_df.day_3.shift(1)
full_df['day_5'] = full_df.day_4.shift(1)
full_df['day_6'] = full_df.day_5.shift(1)

In [ ]:
# Create target and features
y = full_df['change'].loc[6:].tolist()
x = full_df.drop(['change', 'date', 'rate'],axis=1).loc[6:].values.tolist()

In [ ]:
# Create the training and test sets
X, Y = shuffle(x,y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# Use SVM from library to create model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Use linear kernel as points clearly show linear separation
clf = SVC(kernel='linear', random_state = 42)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
for i in range(len(y_pred)):
    print('pred', y_pred[i], 'act' , y_test[i])